## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,59.81,61,45,9.48,scattered clouds
1,1,Mataura,NZ,-46.1927,168.8643,59.49,40,11,7.70,few clouds
2,2,Rikitea,PF,-23.1203,-134.9692,78.76,71,100,10.11,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,84.90,62,49,6.13,scattered clouds
4,4,Jamestown,US,42.0970,-79.2353,22.71,63,100,11.50,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,78.76,71,100,10.11,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,84.90,62,49,6.13,scattered clouds
7,7,Georgetown,MY,5.4112,100.3354,82.33,81,20,3.44,few clouds
10,10,Arraial Do Cabo,BR,-22.9661,-42.0278,75.20,89,83,17.20,broken clouds
21,21,Terenos,BR,-20.4422,-54.8603,76.71,88,0,3.44,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                169
City                   169
Country                169
Lat                    169
Lng                    169
Max Temp               169
Humidity               169
Cloudiness             169
Wind Speed             169
Current Description    169
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.76,overcast clouds,-23.1203,-134.9692,
3,Hermanus,ZA,84.90,scattered clouds,-34.4187,19.2345,
7,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
10,Arraial Do Cabo,BR,75.20,broken clouds,-22.9661,-42.0278,
21,Terenos,BR,76.71,clear sky,-20.4422,-54.8603,
26,Kapaa,US,78.78,moderate rain,22.0752,-159.3190,
28,Tabou,CI,76.80,clear sky,4.4230,-7.3528,
33,Tondano,ID,80.69,broken clouds,1.3038,124.9112,
36,Hobart,AU,87.26,clear sky,-42.8794,147.3294,
37,Sinnamary,GF,76.03,overcast clouds,5.3833,-52.9500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df["Hotel Name"] == "", ["Hotel Name"]] = np.nan
clean_hotel_df = hotel_df.dropna()

print(clean_hotel_df.count())
clean_hotel_df.head(10)

City                   165
Country                165
Max Temp               165
Current Description    165
Lat                    165
Lng                    165
Hotel Name             165
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.76,overcast clouds,-23.1203,-134.9692,People ThankYou
3,Hermanus,ZA,84.90,scattered clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
10,Arraial Do Cabo,BR,75.20,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
21,Terenos,BR,76.71,clear sky,-20.4422,-54.8603,Rancho Pitzer
26,Kapaa,US,78.78,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
28,Tabou,CI,76.80,clear sky,4.4230,-7.3528,hotêl doufoulougou
33,Tondano,ID,80.69,broken clouds,1.3038,124.9112,Villa Bella
36,Hobart,AU,87.26,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
37,Sinnamary,GF,76.03,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} \N{DEGREE SIGN}F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))